## Race condition

In [1]:
import logging
import threading
import time
import concurrent.futures

format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(1)   # simulates a delay during the computation
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

start = time.perf_counter()

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.update, index)
logging.info("Testing update. Ending value is %d.", database.value)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')

11:42:28: Testing update. Starting value is 0.
11:42:28: Thread 0: starting update
11:42:28: Thread 1: starting update
11:42:29: Thread 1: finishing update
11:42:29: Thread 0: finishing update
11:42:29: Testing update. Ending value is 1.


Finished in 1.02 second(s)


## Locks

In [2]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def update(self, name):
        logging.info("Thread %s: starting update", name)
        local_copy = self.value
        local_copy += 1
        time.sleep(1)
        self.value = local_copy
        logging.info("Thread %s: finishing update", name)

    def locked_update(self, name):
        logging.info("Thread %s: starting update", name)
        logging.debug("Thread %s about to lock", name)
        with self._lock:
            logging.debug("Thread %s has lock", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(1)
            self.value = local_copy
            logging.debug("Thread %s about to release lock", name)
        logging.debug("Thread %s after release", name)
        logging.info("Thread %s: finishing update", name)


logging.getLogger().setLevel(logging.DEBUG)
start = time.perf_counter()

database = FakeDatabase()
logging.info("Testing update. Starting value is %d.", database.value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(database.locked_update, index)
logging.info("Testing update. Ending value is %d.", database.value)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,2)} second(s)')


11:44:00: Testing update. Starting value is 0.
11:44:00: Thread 0: starting update
11:44:00: Thread 0 about to lock
11:44:00: Thread 0 has lock
11:44:00: Thread 1: starting update
11:44:00: Thread 1 about to lock
11:44:01: Thread 0 about to release lock
11:44:01: Thread 0 after release
11:44:01: Thread 1 has lock
11:44:01: Thread 0: finishing update
11:44:02: Thread 1 about to release lock
11:44:02: Thread 1 after release
11:44:02: Thread 1: finishing update
11:44:02: Testing update. Ending value is 2.


Finished in 2.03 second(s)
